In [3]:
import aqt

from anki import scheduler_pb2

from IPython import display
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import scipy

import datetime, itertools, os, sys, time

In [4]:
col = mw.col
browser = aqt.dialogs._dialogs['Browser'][1]
table = browser.table
model = table._model

In [5]:
reviews = [model._state.get_review(item) for item in model._items]
cards = [model._state.get_card(item) for item in model._items]
notes = [model._state.get_note(item) for item in model._items]

In [6]:
from pylib.anki.cards import Card
from pylib.anki.notes import Note

def card_repr(self):
    return f"<Card: id={self.id}, type={self.type}, queue={self.queue}, due={self.due}, factor={self.factor}, reps={self.reps}, lapses={self.lapses}>"

def note_repr(self):
    return f"<Note: id={self.id}, fields={self.fields}, tags={self.tags}>"

cards[0].__class__.__repr_orig = cards[0].__class__.__repr__
notes[0].__class__.__repr_orig = notes[0].__class__.__repr__
cards[0].__class__.__repr__ = card_repr
notes[0].__class__.__repr__ = note_repr

In [7]:
CardAnswer = scheduler_pb2.CardAnswer

In [8]:
CardAnswer.GOOD

2

1674684058.506 GET /_anki/legacyPageData
1674684058.506 GET /_anki/legacyPageData
1674684058.517 GET /_anki/css/webview.css
1674684058.518 GET /_anki/css/deckbrowser.css
1674684058.519 GET /_anki/js/webview.js
1674684058.519 GET /_anki/js/vendor/jquery.min.js
1674684058.519 GET /_anki/js/deckbrowser.js
1674684058.520 GET /_anki/js/vendor/jquery-ui.min.js
1674684058.524 GET /_anki/css/webview.css
1674684058.524 GET /_anki/css/toolbar.css
1674684058.525 GET /_anki/js/webview.js
1674684058.525 GET /_anki/css/toolbar-bottom.css
1674684058.526 GET /_anki/imgs/gears.svg
1674684058.527 GET /_anki/js/vendor/jquery.min.js
1674684477.993 GET /_anki/legacyPageData
1674684477.994 GET /_anki/legacyPageData
1674684478.007 GET /_anki/css/webview.css
1674684478.008 GET /_anki/css/deckbrowser.css
1674684478.008 GET /_anki/js/webview.js
1674684478.009 GET /_anki/js/vendor/jquery.min.js
1674684478.011 GET /_anki/js/vendor/jquery-ui.min.js
1674684478.011 GET /_anki/js/deckbrowser.js
1674684478.014 GET /_a

In [126]:
def build_answer(card, rating, seconds_taken, answered_at = None):
    if answered_at is None:
        answered_at = datetime.datetime.now()
    if type(answered_at) == datetime.datetime:
        answered_at = int(answered_at.timestamp() * 1000)
    
    ms_taken = int(seconds_taken * 1000)
    states = model.col._backend.get_scheduling_states(card.id)
    
    if type(rating) == int:
        if rating == 1:
            rating = CardAnswer.AGAIN
        elif rating == 2:
            rating = CardAnswer.HARD
        elif rating == 3:
            rating = CardAnswer.GOOD
        elif rating == 4:
            rating = CardAnswer.EASY
        else:
            raise Exception("invalid int rating")
    
    if rating == CardAnswer.AGAIN:
        new_state = states.again
    elif rating == CardAnswer.HARD:
        new_state = states.hard
    elif rating == CardAnswer.GOOD:
        new_state = states.good
    elif rating == CardAnswer.EASY:
        new_state = states.easy
    else:
        raise Exception("invalid rating")
        
    answer = CardAnswer(
        card_id=card.id,
        current_state=states.current,
        new_state=states.again,
        rating=rating,
        answered_at_millis=answered_at,
        milliseconds_taken=ms_taken,
    )
    return answer

def answer_card(answer):
    model.col._backend.answer_card_raw(answer.SerializeToString())

In [6]:
card = table.get_single_selected_card()
display.HTML(card.note().fields[0])

In [10]:
col.db.execute("""
SELECT *
FROM sqlite_master 
WHERE name = 'notes';
""")

[['table',
  'notes',
  'notes',
  3,
  'CREATE TABLE notes (    id              integer primary key,    guid            text not null,    mid             integer not null,    mod             integer not null,    usn             integer not null,    tags            text not null,    flds            text not null,    sfld            integer not null,    csum            integer not null,    flags           integer not null,    data            text not null)']]

In [11]:
note = table.get_single_selected_card().note()
print(note.__dict__)

{'col': <weakproxy at 0x7fe7a8226540 to Collection at 0x7fe7a8228640>, 'id': 1659117997640, 'guid': 'I3m~1vi&w(', 'mid': 1539794822805, 'mod': 1669093521, 'usn': 1843, 'tags': ['ann:hard', 'ECET-06.02-TrigonometricIntegralsAndSubstitutions'], 'fields': ['&nbsp;ECET-06.02-Xc017', '', ''], '_fmap': {'Front': (0, {'name': 'Front', 'ord': 0, 'sticky': False, 'rtl': False, 'font': 'Liberation Sans', 'size': 20, 'description': '', 'plainText': False, 'collapsed': False, 'media': []}), 'Back': (1, {'name': 'Back', 'ord': 1, 'sticky': False, 'rtl': False, 'font': 'Arial', 'size': 20, 'description': '', 'plainText': False, 'collapsed': False, 'media': []}), 'Annotations': (2, {'name': 'Annotations', 'ord': 2, 'sticky': False, 'rtl': False, 'font': 'Liberation Sans', 'size': 20, 'description': '', 'plainText': False, 'collapsed': False, 'media': []})}}


In [ ]:
note.

In [20]:
n.tags = note.tags
n.fields[0] = note.fields[0]
n.
n

<Note: id=0, fields=['&nbsp;ECET-06.02-Xc017', '', ''], tags=['ann:hard', 'ECET-06.02-TrigonometricIntegralsAndSubstitutions']>

In [18]:
n = col.new_note(m)
n

<Note: id=0, fields=['', '', ''], tags=[]>

In [16]:
m = col.models.get(note.note_type()['id'])
m

{'id': 1539794822805,
 'name': 'PWeave',
 'type': 0,
 'mod': 1670621939,
 'usn': 0,
 'sortf': 0,
 'did': 1535127937393,
 'tmpls': [{'name': 'Card 1',
   'ord': 0,
   'qfmt': '{{Front}}\n',
   'afmt': '{{FrontSide}}\n\n<hr id=answer>\n\n{{Back}}\n\n<hr id=annotations>\n\n{{Annotations}}',
   'bqfmt': '',
   'bafmt': '',
   'did': None,
   'bfont': '',
   'bsize': 0}],
 'flds': [{'name': 'Front',
   'ord': 0,
   'sticky': False,
   'rtl': False,
   'font': 'Liberation Sans',
   'size': 20,
   'description': '',
   'plainText': False,
   'collapsed': False,
   'media': []},
  {'name': 'Back',
   'ord': 1,
   'sticky': False,
   'rtl': False,
   'font': 'Arial',
   'size': 20,
   'description': '',
   'plainText': False,
   'collapsed': False,
   'media': []},
  {'name': 'Annotations',
   'ord': 2,
   'sticky': False,
   'rtl': False,
   'font': 'Liberation Sans',
   'size': 20,
   'description': '',
   'plainText': False,
   'collapsed': False,
   'media': []}],
 'css': '@font-face {\n  f

In [212]:
answered_at = datetime.datetime(2022, 8, 6)

In [213]:
rating = 3 # CardAnswer.GOOD
seconds_taken = 0

browser = aqt.dialogs._dialogs['Browser'][1]
table = browser.table
card = table.get_single_selected_card()
print(f"{answered_at}\n{rating}\n{card}\n")
display.HTML(card.note().fields[0])

2022-08-06 00:00:00
3
<Card: id=1580335330401, type=1, queue=1, due=1674248228, factor=0, reps=3, lapses=0>



In [222]:
rating = 2 # CardAnswer.HARD
seconds_taken = 0

browser = aqt.dialogs._dialogs['Browser'][1]
table = browser.table
card = table.get_single_selected_card()
print(f"{answered_at}\n{rating}\n{card}\n")
display.HTML(card.note().fields[0])

2022-08-05 00:00:00
1
<Card: id=1605842510306, type=1, queue=1, due=1674065626, factor=0, reps=1, lapses=0>



In [200]:
rating = 1 # CardAnswer.AGAIN
seconds_taken = 0

browser = aqt.dialogs._dialogs['Browser'][1]
table = browser.table
card = table.get_single_selected_card()
print(f"{answered_at}\n{rating}\n{card}\n")
display.HTML(card.note().fields[0])

2022-08-07 00:00:00
1
<Card: id=1605842610989, type=0, queue=0, due=1177192, factor=0, reps=0, lapses=0>



In [ ]:
answer_card(build_answer(card=card, rating=rating, seconds_taken=seconds_taken, answered_at = answered_at))

In [220]:
tmp_review = table.get_current_review()
tmp_review.button_chosen = 2
tmp_review

id: 1659852000008
cid: 1580335331679
usn: -1
button_chosen: 2
interval: -60
last_interval: -60

In [221]:
col.update_revlog_entry(tmp_review)

mtime: true
revlog_entry: true
browser_table: true

1674251811.617 POST /_anki/completeTag
JS error /_anki/legacyPageData?id=139642062653184:0 ResizeObserver loop limit exceeded
1674251811.767 POST /_anki/completeTag
1674251811.942 POST /_anki/completeTag
1674251816.445 POST /_anki/completeTag
JS error /_anki/legacyPageData?id=139642062653184:0 ResizeObserver loop limit exceeded
1674251817.552 POST /_anki/completeTag
1674251817.768 POST /_anki/completeTag
1674251818.154 POST /_anki/completeTag
1674251818.406 POST /_anki/completeTag
1674251818.474 POST /_anki/completeTag
1674251818.714 POST /_anki/completeTag
1674251818.851 POST /_anki/completeTag
1674251819.114 POST /_anki/completeTag
1674251819.368 POST /_anki/completeTag
1674251819.506 POST /_anki/completeTag
1674251819.650 POST /_anki/completeTag
1674251819.890 POST /_anki/completeTag
1674251820.017 POST /_anki/completeTag
1674251820.178 POST /_anki/completeTag
1674251820.625 POST /_anki/completeTag
1674251820.760 POST /_anki/completeTag
1674251820.921 POST /_anki/completeTag
1674251

In [221]:
col.db.commit()

1674066256.067 GET /_anki/legacyPageData
1674066256.068 GET /_anki/legacyPageData
1674066256.081 GET /_anki/css/webview.css
1674066256.083 GET /_anki/css/toolbar.css
1674066256.084 GET /_anki/css/toolbar-bottom.css
1674066256.084 GET /_anki/css/webview.css
1674066256.085 GET /_anki/js/webview.js
1674066256.086 GET /_anki/js/vendor/jquery.min.js
1674066256.089 GET /_anki/css/deckbrowser.css
1674066256.089 GET /_anki/js/webview.js
1674066256.090 GET /_anki/js/vendor/jquery-ui.min.js
1674066256.090 GET /_anki/js/vendor/jquery.min.js
1674066256.091 GET /_anki/js/deckbrowser.js
1674066256.091 GET /_anki/imgs/gears.svg
1674066403.889 POST /_anki/completeTag
JS error /_anki/legacyPageData?id=140349734168896:0 ResizeObserver loop limit exceeded
1674066404.021 POST /_anki/completeTag
1674066404.366 POST /_anki/completeTag
1674066404.531 POST /_anki/completeTag
1674066404.675 POST /_anki/completeTag
1674066404.766 POST /_anki/completeTag
1674066409.397 POST /_anki/completeTag
JS error /_anki/leg

In [23]:
print(col.db.execute(f'''select * from reviews where cid = {card.id};'''))
print(col.db.execute(f'''select * from revlog where cid = {card.id};'''))

[[1605845472630, 1605841841781, -1, 1, -60, 0, 0, 109467, 0, '', '', 0], [1605847402309, 1605841841781, -1, 4, 2, -60, 2500, 255085, 0, '', '', 0], [1659592800001, 1605841841781, -1, 3, 6, 2, 2500, 0, 1, '', '', 0], [1659592800004, 1605841841781, -1, 3, -600, 6, 2300, 0, 1, '', '', 0]]
[[1605845472630, 1605841841781, -1, 1, -60, 0, 0, 109467, 0], [1605847402309, 1605841841781, -1, 4, 2, -60, 2500, 255085, 0], [1659592800001, 1605841841781, -1, 3, 6, 2, 2500, 0, 1], [1659592800004, 1605841841781, -1, 3, -600, 6, 2300, 0, 1]]
***** SqliteStorage.remove_revlog_entry(id=RevlogId(1659592800004))
Editor.onBridgeCmd(): ignored late review blur because:
Editor.onBridgeCmd(): self.review is unset.


In [19]:
print(col.db.execute(f'''select * from reviews where cid = {1673993755760};'''))
print(col.db.execute(f'''select * from revlog where cid = {1673993755760};'''))

[[1659592800000, 1673993755760, -1, 3, -600, 0, 0, 0, 0, '', '', 0]]
[[1659592800000, 1673993755760, -1, 3, -600, 0, 0, 0, 0]]


In [13]:
answered_at = datetime.datetime(2022, 8, 4)
rating = CardAnswer.GOOD
seconds_taken = 0

browser = aqt.dialogs._dialogs['Browser'][1]
table = browser.table
card = table.get_single_selected_card()
print(f"{answered_at}\n{rating}\n{card}\n{card.note().fields[0]}")

2022-08-04 00:00:00
2
<Card: id=1605841841781, type=3, queue=1, due=1674002123, factor=2300, reps=7, lapses=1>

<div class="container">
  <div class="row">
    <div class="col-md-12 twelve columns">

<p>CET-07.01-03 Exercise:</p>
<h4>Techniques of Integration</h4>
<h5>Integration by Parts</h5>
<p>Temporary: work out of CET textbook.</p>

      </div>
  </div>
</div>

sql: SELECT id,   nid,   did,   ord,   cast(mod AS integer),   usn,   type,   queue,   due,   cast(ivl AS integer),   factor,   reps,   lapses,   left,   odue,   odid,   flags,   data FROM cards where id = 1605841841781
sql: SELECT id,   guid,   mid,   mod,   usn,   tags,   flds,   cast(sfld AS text),   csum FROM notes where id = 1605841841780


In [14]:
answer_card(
    build_answer(
        card=card,
        rating=rating,
        seconds_taken=seconds_taken,
        answered_at = answered_at
    )
)

sql: SELECT id,   nid,   did,   ord,   cast(mod AS integer),   usn,   type,   queue,   due,   cast(ivl AS integer),   factor,   reps,   lapses,   left,   odue,   odid,   flags,   data FROM cards where id = 1605841841781
sql: SELECT id,   name,   mtime_secs,   usn,   common,   kind FROM decks where id = 1605841725442
sql: SELECT id,   name,   mtime_secs,   usn,   common,   kind FROM decks where id = 1535127937393
sql: SELECT id,   name,   mtime_secs,   usn,   config FROM deck_config where id = 1
***** SchedulerService.answer_card(input=CardAnswer { card_id: 1605841841781, current_state: Some(SchedulingState { custom_data: None, value: Some(Filtered(Filtered { value: Some(Rescheduling(ReschedulingFilter { original_state: Some(Normal { value: Some(Relearning(Relearning { review: Some(Review { scheduled_days: 1, elapsed_days: 1, ease_factor: 2.3, lapses: 1, leeched: false }), learning: Some(Learning { remaining_steps: 1, scheduled_secs: 600 }) })) }) })) })) }), new_state: Some(SchedulingS

In [26]:
col.db.commit()

***** SqliteStorage.commit_trx()


In [ ]:
c = table.get_single_selected_card()
print(c, c.note())

In [15]:
print(col.db.execute(f'''select * from reviews where cid = {card.id};'''))
print(col.db.execute(f'''select * from revlog where cid = {card.id};'''))

[[1605845472630, 1605841841781, -1, 1, -60, 0, 0, 109467, 0, '', '', 0], [1605847402309, 1605841841781, -1, 4, 2, -60, 2500, 255085, 0, '', '', 0], [1659592800000, 1605841841781, -1, 3, -600, -600, 2300, 0, 2, '', '', 0]]
[[1605845472630, 1605841841781, -1, 1, -60, 0, 0, 109467, 0], [1605847402309, 1605841841781, -1, 4, 2, -60, 2500, 255085, 0], [1659592800000, 1605841841781, -1, 3, -600, -600, 2300, 0, 2]]


In [ ]:
print(col.db.execute(f'''select * from reviews where cid = {card.id};'''))

In [20]:
table.get_single_selected_card().note()

<Note: id=1659650913953, fields=['CET-07.04-Xc007', '', ''], tags=['0-Textbooks::CalculusEarlyTrancendentals::C07-TechniquesOfIntegration::CET-07.04-IntegrationOfRationalFunctionsByPartialFractions']>

1673994100.121 GET /_anki/legacyPageData
1673994100.122 GET /_anki/legacyPageData
1673994100.138 GET /_anki/css/webview.css
1673994100.139 GET /_anki/css/deckbrowser.css
1673994100.140 GET /_anki/js/webview.js
1673994100.140 GET /_anki/js/vendor/jquery.min.js
1673994100.141 GET /_anki/js/vendor/jquery-ui.min.js
1673994100.142 GET /_anki/js/deckbrowser.js
1673994100.142 GET /_anki/imgs/gears.svg
1673994100.147 GET /_anki/css/webview.css
1673994100.148 GET /_anki/js/vendor/jquery.min.js
1673994100.148 GET /_anki/js/webview.js
1673994100.149 GET /_anki/css/toolbar.css
1673994100.149 GET /_anki/css/toolbar-bottom.css
Editor.onBridgeCmd(): ignored late review blur because:
Editor.onBridgeCmd(): self.review is unset.
Editor.onBridgeCmd(): ignored late review blur because:
Editor.onBridgeCmd(): self.review is unset.
Editor.onBridgeCmd(): ignored late review blur because:
Editor.onBridgeCmd(): self.review is unset.
1673994166.816 GET /_anki/pages/card-info.html
1673994166.826 GET /_anki/pages/

In [ ]:
col.db.execute("""
SELECT name 
FROM sqlite_master 
WHERE type = 'index';
""")

In [7]:
col.db.execute("""CREATE INDEX IF NOT EXISTS ix_revlog_cid ON revlog (cid);""")
col.db.execute("""CREATE INDEX IF NOT EXISTS ix_review_notes_feedback ON review_notes (feedback);""")
col.db.execute("""CREATE INDEX IF NOT EXISTS ix_review_notes_tags ON review_notes (tags);""")
col.db.execute("""CREATE INDEX IF NOT EXISTS ix_notes_tags ON notes (tags);""")
col.db.execute("""CREATE INDEX IF NOT EXISTS ix_notes_tags ON notes (tags);""")
col.db.execute("""CREATE INDEX IF NOT EXISTS idx_fields_name_ntid ON fields (name, ntid);""")
col.db.execute("""CREATE INDEX IF NOT EXISTS idx_templates_name_ntid ON templates (name, ntid);""")
col.db.execute("""CREATE INDEX IF NOT EXISTS idx_notetypes_name ON notetypes (name);""")
col.db.execute("""CREATE INDEX IF NOT EXISTS idx_decks_name ON decks (name);""")


sql: CREATE INDEX IF NOT EXISTS ix_revlog_cid ON revlog (cid);
sql: CREATE INDEX IF NOT EXISTS ix_review_notes_feedback ON review_notes (feedback);
sql: CREATE INDEX IF NOT EXISTS ix_review_notes_tags ON review_notes (tags);
sql: CREATE INDEX IF NOT EXISTS ix_notes_tags ON notes (tags);
sql: CREATE INDEX IF NOT EXISTS ix_notes_tags ON notes (tags);
sql: CREATE INDEX IF NOT EXISTS idx_fields_name_ntid ON fields (name, ntid);
sql: CREATE INDEX IF NOT EXISTS idx_templates_name_ntid ON templates (name, ntid);
sql: CREATE INDEX IF NOT EXISTS idx_notetypes_name ON notetypes (name);
sql: CREATE INDEX IF NOT EXISTS idx_decks_name ON decks (name);


[]

In [ ]:
col.db.execute("""select tags from review_notes where tags != ''""")

In [ ]:
col.db.execute("""select distinct n.id from notes n where (n.tags regexp '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*') order by n.sfld collate nocase asc""")b

In [ ]:
col.db.execute("""
select notes.id from (
    notes inner join cards on notes.id == cards.nid
) left join reviews on cards.id == reviews.cid
where (
    (notes.tags like '%000%')
    or
    (reviews.tags like '%feedback%')
)
""")

In [ ]:
col.db.execute("""
select distinct n.id from notes n inner join cards c on c.nid == n.id left join revlog as rl on c.id == rl.cid inner join review_notes as r on r.id == rl.id
where regexp(n.tags, '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*') or regexp(r.tags, '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*')
""")

In [ ]:
col.db.execute("""
select notes.id from (
    notes inner join cards on notes.id == cards.nid
) left join reviews on cards.id == reviews.cid
where (
    (notes.tags like '%000-Textbook-CalculusConceptsAndContexts::C02-LimitsAndDerivatives::SX-Review%')
    or
    (reviews.tags like '%000-Textbook-CalculusConceptsAndContexts::C02-LimitsAndDerivatives::SX-Review%')
)
""")

In [ ]:
col.db.execute("""
SELECT notes.id FROM
notes
INNER JOIN cards ON notes.id == cards.nid
LEFT JOIN reviews ON cards.id == reviews.cid
where (
    (notes.tags like '%000%')
    or
    (reviews.tags like '%feedback%')
)
""")

In [ ]:
col.db.execute("""
select notes.id from
notes inner join (
    cards left join reviews on cards.id == reviews.cid
    cards inner join reviews on cards.id == reviews.cid
    union
    
) on notes.id == cards.nid
where (
    (notes.tags like '%000-Textbook-CalculusConceptsAndContexts::C02-LimitsAndDerivatives::SX-Review%')
    or
    (reviews.tags like '%000-Textbook-CalculusConceptsAndContexts::C02-LimitsAndDerivatives::SX-Review%')
)
""")

In [ ]:
col.db.execute("""
select notes.id from
notes inner join (
    cards left join reviews on cards.id == reviews.cid
) on notes.id == cards.nid
where (
    (notes.tags like '%000-Textbook-CalculusConceptsAndContexts::C02-LimitsAndDerivatives::SX-Review%')
    or
    (reviews.tags like '%000-Textbook-CalculusConceptsAndContexts::C02-LimitsAndDerivatives::SX-Review%')
)
""")

In [ ]:
col.db.execute("""
SELECT DISTINCT n.id FROM notes AS n
  INNER JOIN cards AS c ON
    n.id == c.nid
  LEFT JOIN revlog AS rl ON
    c.id == rl.cid
  INNER JOIN review_notes AS r ON
    r.id == rl.id
    AND (
      n.tags regexp '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*'
    )
  ORDER BY n.sfld COLLATE NOCASE ASC
""")

In [ ]:
col.db.execute("""
select n.id
    from
        notes as n
    inner join
        cards as c
        on n.id == c.nid
    left join
        reviews as r
        on c.id == r.cid
    where (
        (n.tags regexp '.*000.*')
        or
        (r.tags regexp '.*feedback.*')
    )
""")

In [ ]:
col.db.execute("""
select
        r.id
    from reviews r
    inner join cards c on r.cid == r.id
    inner join notes n on c.nid == n.id
    where (
        (n.tags regexp '.*000.*')
        or
        (r.tags regexp '.*feedback.*')
    )
""")

In [ ]:
col.db.execute("""
select
        distinct n.id
    from cards as c
    left join reviews as r
        on c.id==r.cid
    inner join notes as n
        on c.nid==n.id
    where (
        (n.tags regexp '.*000.*')
        or
        (r.tags regexp '.*feedback.*')
    )
""")

In [ ]:
 #col.db.execute("""select n.id from notes n where (n.tags regexp '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*') order by n.sfld collate nocase asc""")
col.db.execute("""select distinct n.id from cards as c left join reviews as r on c.id=r.cid inner join notes as n on c.nid=n.id where and (n.tags regexp '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*' or r.tags regexp '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*') order by n.sfld collate nocase asc""")

In [ ]:
 #col.db.execute("""select n.id from notes n where (n.tags regexp '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*') order by n.sfld collate nocase asc""")
col.db.execute("""
   select distinct n.id
      from
          notes n
      inner join
          cards c
        on cards.nid == notes.id
    
      where r.cid=c.id and c.nid=n.id and (n.tags regexp '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*' or r.tags regexp '(?i).* 000\-Textbook\-CalculusConceptsAndContexts::C03\-DifferentiationRules::SX\-Review(::| ).*') order by n.sfld collate nocase asc
""")

In [ ]:
browser = aqt.dialogs._dialogs['Browser'][1]
table = browser.table
model = table._model

In [230]:
col.db.execute("""select * from revlog where cid == 1659061591875""")

[[1659295928602, 1659061591875, -1, 1, -60, -60, 0, 11454, 0],
 [1669072141900, 1659061591875, -1, 1, -60, -60, 0, 7866, 0]]

1674070456.005 GET /_anki/legacyPageData
1674070456.005 GET /_anki/legacyPageData
1674070456.017 GET /_anki/css/webview.css
1674070456.017 GET /_anki/css/deckbrowser.css
1674070456.018 GET /_anki/js/webview.js
1674070456.019 GET /_anki/js/vendor/jquery.min.js
1674070456.021 GET /_anki/js/vendor/jquery-ui.min.js
1674070456.021 GET /_anki/js/deckbrowser.js
1674070456.024 GET /_anki/css/webview.css
1674070456.025 GET /_anki/css/toolbar.css
1674070456.026 GET /_anki/css/toolbar-bottom.css
1674070456.026 GET /_anki/js/webview.js
1674070456.027 GET /_anki/js/vendor/jquery.min.js
1674070456.027 GET /_anki/imgs/gears.svg


In [232]:
col.db.execute("""select * from review_notes where id in (1659295928602, 1669072141900)""")

[[1659295928602,
  'Trouble: Dropped factor of 5 in common denominator. Looks like this occurred because I temporarily mistook a&nbsp;\\(+\\)&nbsp;sign for an&nbsp;\\(=\\)&nbsp;sign.',
  ' .feedback::trouble::algebra::conflation::symbol-swap .feedback::trouble::algebra::omission::dropped-factor ',
  1659295928,
  'V'],
 [1669072141900,
  'Technique: substitution of square root of a function<br>Trouble: Dropped factor of 5 in common denominator. Looks like this occurred because I temporarily mistook a&nbsp;\\(+\\)&nbsp;sign for an&nbsp;\\(=\\)&nbsp;sign.',
  ' .feedback::technique::substitution .feedback::trouble::algebra::conflation::symbol-swap .feedback::trouble::algebra::omission::dropped-factor ',
  1669072141,
  '']]

In [233]:
col.db.execute("""select * from revlog inner join review_notes on revlog.id == review_notes.id where cid == 1659061591875""")

[[1659295928602,
  1659061591875,
  -1,
  1,
  -60,
  -60,
  0,
  11454,
  0,
  1659295928602,
  'Trouble: Dropped factor of 5 in common denominator. Looks like this occurred because I temporarily mistook a&nbsp;\\(+\\)&nbsp;sign for an&nbsp;\\(=\\)&nbsp;sign.',
  ' .feedback::trouble::algebra::conflation::symbol-swap .feedback::trouble::algebra::omission::dropped-factor ',
  1659295928,
  'V'],
 [1669072141900,
  1659061591875,
  -1,
  1,
  -60,
  -60,
  0,
  7866,
  0,
  1669072141900,
  'Technique: substitution of square root of a function<br>Trouble: Dropped factor of 5 in common denominator. Looks like this occurred because I temporarily mistook a&nbsp;\\(+\\)&nbsp;sign for an&nbsp;\\(=\\)&nbsp;sign.',
  ' .feedback::technique::substitution .feedback::trouble::algebra::conflation::symbol-swap .feedback::trouble::algebra::omission::dropped-factor ',
  1669072141,
  '']]

In [234]:
col.db.execute("""select * from reviews where cid == 1659061591875""")

[[1659295928602,
  1659061591875,
  -1,
  1,
  -60,
  -60,
  0,
  11454,
  0,
  'Trouble: Dropped factor of 5 in common denominator. Looks like this occurred because I temporarily mistook a&nbsp;\\(+\\)&nbsp;sign for an&nbsp;\\(=\\)&nbsp;sign.',
  ' .feedback::trouble::algebra::conflation::symbol-swap .feedback::trouble::algebra::omission::dropped-factor ',
  1659295928]]

In [235]:
col.db.execute("""select * from sqlite_schema where name == 'reviews'""")

[['view',
  'reviews',
  'reviews',
  0,
  "CREATE VIEW reviews AS\n  SELECT revlog.id,\n    revlog.cid,\n    revlog.usn,\n    revlog.ease,\n    revlog.ivl,\n    revlog.lastIvl,\n    revlog.factor,\n    revlog.time,\n    revlog.type,\n    review_notes.feedback,\n    review_notes.tags,\n    review_notes.mod\n  FROM revlog\n    INNER JOIN review_notes ON revlog.id = review_notes.id\n  WHERE review_notes.vis IN ('D', 'V')"]]

In [24]:
print(col.db.execute(f'''select * from reviews where id = {1673993755760};'''))

[[1659592800000, 1673993755760, -1, 3, -600, 0, 0, 0, 0, '', '', 0]]
[[1659592800000, 1673993755760, -1, 3, -600, 0, 0, 0, 0]]
1674080078.111 GET /_anki/legacyPageData
1674080078.112 GET /_anki/legacyPageData
1674080078.123 GET /_anki/css/webview.css
1674080078.125 GET /_anki/css/webview.css
1674080078.126 GET /_anki/css/toolbar.css
1674080078.126 GET /_anki/css/toolbar-bottom.css
1674080078.127 GET /_anki/js/webview.js
1674080078.128 GET /_anki/js/vendor/jquery.min.js
1674080078.132 GET /_anki/css/deckbrowser.css
1674080078.133 GET /_anki/js/vendor/jquery.min.js
1674080078.133 GET /_anki/js/vendor/jquery-ui.min.js
1674080078.134 GET /_anki/js/webview.js
1674080078.134 GET /_anki/js/deckbrowser.js
1674080078.136 GET /_anki/imgs/gears.svg


**Source review**

In [105]:
review = table.get_current_review()
rating = review.button_chosen
seconds_taken = review.taken_millis/1000
answered_at = datetime.datetime.fromtimestamp(review.id/1000)

print(review)
print(f"rating: {rating}, seconds_taken: {seconds_taken}, answered_at: {answered_at}")
display.HTML(col.get_card(review.cid).note().fields[0])

id: 1610650851864
cid: 1605841844708
usn: 2108
button_chosen: 4
interval: 2
last_interval: -60
ease_factor: 2500
taken_millis: 1080440

rating: 4, seconds_taken: 1080.44, answered_at: 2021-01-14 12:00:51.864000


**Target card**

In [106]:
browser = aqt.dialogs._dialogs['Browser'][1]
table = browser.table
card = table.get_single_selected_card()
display.HTML(card.note().fields[0])

In [107]:
answer_card(build_answer(card=card, rating=rating, seconds_taken=seconds_taken, answered_at = answered_at))

**Verify source review attributes**

In [108]:
review

id: 1610650851864
cid: 1605841844708
usn: 2108
button_chosen: 4
interval: 2
last_interval: -60
ease_factor: 2500
taken_millis: 1080440

1674246364.865 GET /_anki/legacyPageData
1674246364.866 GET /_anki/legacyPageData
1674246364.963 GET /_anki/css/webview.css
1674246364.964 GET /_anki/css/deckbrowser.css
1674246364.965 GET /_anki/js/webview.js
1674246364.966 GET /_anki/js/vendor/jquery.min.js
1674246364.967 GET /_anki/js/vendor/jquery-ui.min.js
1674246364.968 GET /_anki/js/deckbrowser.js
1674246364.973 GET /_anki/imgs/gears.svg
1674246364.974 GET /_anki/css/webview.css
1674246364.975 GET /_anki/css/toolbar.css
1674246364.975 GET /_anki/css/toolbar-bottom.css
1674246364.977 GET /_anki/js/webview.js
1674246364.977 GET /_anki/js/vendor/jquery.min.js
1674246365.043 GET /_anki/legacyPageData
1674246365.047 GET /_anki/legacyPageData
1674246365.075 GET /_anki/css/webview.css
1674246365.075 GET /_anki/css/deckbrowser.css
1674246365.075 GET /_anki/js/webview.js
1674246365.080 GET /_anki/js/vendor/jquery.min.js
1674246365.082 GET /_anki/js/vendor/jquery-ui.min.js
1674246365.082 GET /_anki/js/deckbrowser.js
1674246365.083 GET /_a

**Transfer attributes to target review**

In [30]:
new_review = table.get_current_review()
new_review.feedback = review.feedback
new_review.tags.extend(review.tags)
new_review

id: 1659679200038
cid: 1580335328424
usn: -1
button_chosen: 2
interval: -60
last_interval: -60
feedback: "Almost missed the obvious: \\(\\ln\\sqrt x = \\tfrac{1}{2}\\ln x\\)."
tags: ".feedback::trouble::obvious::log-laws"

1674245591.079 GET /_anki/legacyPageData
1674245591.080 GET /_anki/legacyPageData
1674245591.092 GET /_anki/css/webview.css
1674245591.095 GET /_anki/css/webview.css
1674245591.096 GET /_anki/css/toolbar.css
1674245591.096 GET /_anki/css/toolbar-bottom.css
1674245591.097 GET /_anki/js/webview.js
1674245591.097 GET /_anki/js/vendor/jquery.min.js
1674245591.100 GET /_anki/css/deckbrowser.css
1674245591.100 GET /_anki/js/webview.js
1674245591.102 GET /_anki/js/vendor/jquery.min.js
1674245591.103 GET /_anki/js/vendor/jquery-ui.min.js
1674245591.104 GET /_anki/js/deckbrowser.js
1674245591.105 GET /_anki/imgs/gears.svg


In [31]:
col.update_revlog_entry(new_review)

mtime: true
revlog_entry: true
browser_table: true

Editor.onBridgeCmd(): ignored late review blur because:
Editor.onBridgeCmd(): self.review is unset.
